# Stroke Detection using EfficientNet-B0

We are doing Binary Classification to detect Strokes from images using the EfficientNet-B0 Model.

## Dataset Import

In [1]:
import kagglehub
danish003_face_images_of_acute_stroke_and_non_acute_stroke_path = kagglehub.dataset_download('danish003/face-images-of-acute-stroke-and-non-acute-stroke')
print('Data source import complete.', danish003_face_images_of_acute_stroke_and_non_acute_stroke_path)

100%|██████████| 36.7M/36.7M [00:04<00:00, 7.77MB/s]

Extracting files...


Data source import complete. C:\Users\shahk\.cache\kagglehub\datasets\danish003\face-images-of-acute-stroke-and-non-acute-stroke\versions\1


## Imports and Setup

In [2]:
import os
from pathlib import Path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models


In [5]:
DATA_DIR = Path(danish003_face_images_of_acute_stroke_and_non_acute_stroke_path + r'\main')
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transform)

val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

val_dataset.dataset.transform = val_transform

BATCH_SIZE = 32
NUM_WORKERS = 2

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
class_names = full_dataset.classes
print(class_names)

['noStroke_data', 'stroke_data']


## EfficientNet B0 Model and Training

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(weights=weights)

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 2)

model = model.to(device)

for param in model.features.parameters():
    param.requires_grad = True

cpu


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\shahk/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:02<00:00, 10.3MB/s]


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=2)

In [ ]:
from copy import deepcopy
import time
from sklearn.metrics import f1_score

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / total, correct / total


def eval_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='weighted')
    return running_loss / total, correct / total, f1


EPOCHS = 15
best_val_acc = 0.0
best_model_wts = deepcopy(model.state_dict())

for epoch in range(EPOCHS):
    start = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, val_f1 = eval_one_epoch(model, val_loader, criterion, device)
    scheduler.step(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = deepcopy(model.state_dict())

    print(
        f"Epoch {epoch+1}/{EPOCHS} | ",
        f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | ",
        f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f} F1: {val_f1:.4f} | ",
        f"Time: {time.time()-start:.1f}s"
    )

    if(train_acc == 1.0 and val_acc == 1.0 and val_f1 == 1.0):
        print("Achieved 100% accuracy on both training and validation sets. Stopping early.")
        break

# Load best weights and save
model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), "efficientnet_b0_stroke_best.pth")
